In [2]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
## First, get the api key
with open('cfb_apikey.txt', 'r') as file:
    apikey = f"Bearer {file.read()}"

In [4]:
# specify base_url and endpoint url
url = "https://api.collegefootballdata.com/drives?seasonType=regular&year=2008&team=Utah"

In [5]:
headers = {
    "Authorization": apikey,
    "Content-Type": "application/json"
}

In [6]:
r = requests.get(url, headers=headers)
print(r.ok)

True


In [7]:
my_dict = r.json()

In [10]:
df = pd.DataFrame(my_dict)
df.columns

Index(['offense', 'offense_conference', 'defense', 'defense_conference',
       'game_id', 'id', 'drive_number', 'scoring', 'start_period',
       'start_yardline', 'start_yards_to_goal', 'start_time', 'end_period',
       'end_yardline', 'end_yards_to_goal', 'end_time', 'elapsed', 'plays',
       'yards', 'drive_result', 'is_home_offense', 'start_offense_score',
       'start_defense_score', 'end_offense_score', 'end_defense_score'],
      dtype='object')

In [12]:
columns_wanted = ['offense', 'defense', 'scoring', 'elapsed', 'plays', 'start_yards_to_goal', 'yards', 'drive_result']
df[columns_wanted]

,offense,defense,scoring,elapsed,plays,start_yards_to_goal,yards,drive_result
0,Michigan,Utah,False,"{'minutes': 1, 'seconds': 32}",3,71,2,PUNT
1,Michigan,Utah,True,"{'minutes': 2, 'seconds': 8}",4,26,26,PASSING TD
2,Utah,Michigan,True,"{'minutes': 3, 'seconds': 42}",8,75,75,RUSHING TD
3,Michigan,Utah,True,"{'minutes': 2, 'seconds': 20}",6,50,17,FG GOOD
4,Utah,Michigan,True,"{'minutes': 6, 'seconds': 14}",14,77,66,FG GOOD
...,...,...,...,...,...,...,...,...
309,Utah,BYU,True,"{'seconds': 4, 'minutes': 0}",1,4,4,PASSING TD
310,BYU,Utah,False,"{'minutes': 3, 'seconds': 58}",8,75,56,INT
311,Utah,BYU,True,"{'minutes': 5, 'seconds': 24}",9,29,29,PASSING TD
312,BYU,Utah,False,"{'seconds': 32, 'minutes': 0}",4,67,13,INT
